# Dynamic sensitivity analysis
ok so we have these components
* initial conditions (in `initial_conditions.txt`)
* x<sub>i</sub>(t) for all i in [1, 24] (in `x(t).csv`)
* sensitivity matrix (sbar) as a function of k<sub>j</sub> and x<sub>i</sub> (in `sensitivity_matrix` pickle file)
* k<sub>j</sub> for all j [1, 64] (in `reaction_coefficients.txt`)

Using these components, we can calculate equations (21) and (22)

### load all these bad bois

In [71]:
# get x(t) for all t
xt = {'x{}'.format(i+1): [] for i in range(24)}
with open('./x(t).csv', 'r') as o:
    header = False
    for line in o:
        if not header:
            header = True
            continue
        vals = line.split(',')[1:]
        numvals = [float(x) for x in vals]
        for i in range(len(numvals)):
            xt['x{}'.format(i+1)].append(numvals[i])
        
# get the sensitivity matrix
import pickle
sbar = pickle.load(open('./sensitivity_matrix', 'rb'))

# get all of the ks
ks = {}
with open('./reaction_coefficients.txt', 'r') as o:
    for line in o:
        k = line.split('=')[0]
        ks[k] = sp.sympify(line.split('=')[1].replace('\n', '').strip()).evalf()
    
print(len(sbar))
print(len(sbar[0]))

24
63


## Dynamic sensitivity analysis with multiple variables
This is equation 22 of the paper
![Equation 22](./equation22.png)

In [65]:
import sympy as sp
# calc sbar 
def s_bar(i, j, k):
    # get all the substitute values for the expression
    # get the row of xt for the k
    tosub = {}
    for key in xt.keys():
        tosub[key] = xt[key][k]
    # get all the ks
    for key in ks.keys():
        tosub[key] = ks[key]
    # get the i j entry of sbar
    s = sbar[i][j] 
    # substitute and eval
    a = s.evalf(subs=tosub)
    b = float(a)
    return b

In [66]:
# do the intermost summation
def inner_sum(k, j, i_range):
    tosum = []
    for i in range(i_range):
        tosum.append(s_bar(i, j, k) ** 2)
    return sum(tosum)

In [67]:
# do the outer summation
def outer_sum(N, j, i_range):
    tosum = []
    for k in range(N):
        tosum.append(inner_sum(k, j, i_range))
    return sum(tosum)

In [68]:
# do the 1/n and the sqrt
import numpy as np
def eqn22(N, j, i_range):
    return (1/N) * np.sqrt(outer_sum(N, j, i_range))

In [70]:
# do the sensitivity analysis
i_range = len(xt.keys())
N = len(xt['x1'])
j_range = len(ks.keys())
OS = []
for j in range(j_range):
    print('Calculating sensitivity for j {}/{}\r'.format(j, j_range), end='')
    OS.append(eqn22(N, j, i_range))
    

IndexError: list index out of range